In [1]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering import FeatureEngineering
from models import Model1Xgb, Model1Lgb, Model1NN, Model2Linear


%matplotlib inline

print("OK\n")

OK



In [2]:
def geometric_mean(x):
    return np.exp(np.log(x[x>0]).mean())

In [3]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

In [74]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        print(pred)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

In [4]:
#app_train_origin = pd.read_csv('./home-credit-default-risk/exports/app_train.csv')
#app_test_origin =  pd.read_csv('./home-credit-default-risk/exports/app_test.csv')
bureau_origin = pd.read_csv('./home-credit-default-risk/bureau.csv')
bureau_balance_origin = pd.read_csv('./home-credit-default-risk/bureau_balance.csv')
pos_cash_balance_origin = pd.read_csv('./home-credit-default-risk/POS_CASH_balance.csv')
credit_card_balance_origin = pd.read_csv('./home-credit-default-risk/credit_card_balance.csv')
previous_application_origin = pd.read_csv('./home-credit-default-risk/previous_application.csv')
installments_payments_origin = pd.read_csv('./home-credit-default-risk/installments_payments.csv')
# custom data
#app_train_loan_from_hc_only_cash_origin = pd.read_csv('./home-credit-default-risk/datasets/app_train_loan_from_hc_only_cash.csv')
#app_test_loan_from_hc_only_cash_origin = pd.read_csv('./home-credit-default-risk/datasets/app_test_loan_from_hc_only_cash.csv')
#app_train_1000_mean_origin = pd.read_csv('./home-credit-default-risk/exports/train_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
#app_test_1000_mean_origin = pd.read_csv('./home-credit-default-risk/exports/test_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
#app_train_max_installment_ratio_origin = pd.read_csv('./home-credit-default-risk/exports/train_hc_annuity_to_max_installment_ratio.csv')
#app_test_max_installment_ratio_origin = pd.read_csv('./home-credit-default-risk/exports/test_hc_annuity_to_max_installment_ratio.csv')

In [5]:
#app_train = app_train_origin.copy()
#app_test = app_test_origin.copy()
bureau = bureau_origin.copy()
bureau_balance = bureau_balance_origin.copy()
pos_cash_balance = pos_cash_balance_origin.copy()
credit_card_balance = credit_card_balance_origin.copy()
previous_application = previous_application_origin.copy()
installments_payments = installments_payments_origin.copy()
#app_train_1000_mean = app_train_1000_mean_origin.copy()
#app_test_1000_mean = app_test_1000_mean_origin.copy()
#app_train_max_installment_ratio = app_train_max_installment_ratio_origin.copy()
#app_test_max_installment_ratio = app_test_max_installment_ratio_origin.copy()
pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [6]:
df = pd.read_csv(r'./home-credit-default-risk/application_train.csv')
test_df = pd.read_csv(r'./home-credit-default-risk/application_test.csv')
df = df.append(test_df).reset_index()

docs = [_f for _f in df.columns if 'FLAG_DOC' in _f]
live = [_f for _f in df.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]

# general cleaning procedures
df = df[df['CODE_GENDER'] != 'XNA']
df = df[df['AMT_INCOME_TOTAL'] < 20000000] # remove a outlier 117M
# NaN values for DAYS_EMPLOYED: 365.243 -> nan
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True) # set null value
df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True) # set null value

### CODE_GENDER, 
# Categorical features with Binary encode (0 or 1; two categories)
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

### NAME_INCOME_TYPE
df['NAME_INCOME_TYPE'].replace(['Maternity leave'], ['Unemployed'], inplace=True)
df['NAME_INCOME_TYPE'].replace(['Working','Commercial associate','Pensioner','State servant','Unemployed','Student','Businessman','Maternity leave'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
### NAME_FAMILY_STATUS
df['NAME_FAMILY_STATUS'].replace('Unknown', 'Married', inplace=True)
df['NAME_FAMILY_STATUS'].replace(['Married','Single / not married','Civil marriage','Separated','Widow'], [0, 1, 2, 3, 4], inplace=True)

### 
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))

### 列名から不正な文字を削除
for col in df.columns:
    df[col] = df[col].replace('\s', '_', regex=True)
    df[col] = df[col].replace(':', '_', regex=True)
    df[col] = df[col].replace('/', '_', regex=True)


# nullに置き換え
df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].replace(365243, np.nan)

inc_by_org = df[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']

# 特徴量1: 総所得金額を世帯人数で除した値
df['INCOME_div_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']

# 特徴量2: 総所得額を就労期間で除した値
df['INCOME_div_EMPLOYED'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED']

#
df['NEW_CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']

df['NEW_CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']

df['NEW_DOC_IND_KURT'] = df[docs].kurtosis(axis=1)

df['NEW_LIVE_IND_SUM'] = df[live].sum(axis=1)

df['NEW_INC_PER_CHLD'] = df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN'])

df['NEW_INC_BY_ORG'] = df['ORGANIZATION_TYPE'].map(inc_by_org)

df['NEW_EMPLOY_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']

df['NEW_EMPLOY_TO_BIRTH-18_RATIO'] = df['DAYS_EMPLOYED'] / (df['DAYS_BIRTH'] + 18*365)

df['NEW_BIRTH_TO_EMPLOY_RATIO'] = df['DAYS_BIRTH'] / (1 + df['DAYS_EMPLOYED'])

df['NEW_INCOME_TO_ANNUITY_RATIO'] = df['AMT_INCOME_TOTAL'] / (1 + df['AMT_ANNUITY'])

df['NEW_ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / (1 + df['AMT_INCOME_TOTAL'])


# 特徴量3: 外部スコアの平均
df['NEW_EXT_SOURCES_MEDIAN'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].median(axis=1, skipna=True)

df['EXT_SOURCE_mean'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    
df['EXT_SOURCE_max'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max(axis=1)

df['EXT_SOURCE_min'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1)

df['EXT_SOURCE_std'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

df['EXT_SOURCE_count'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].notnull().sum(axis=1)

df['NEW_EXT_SOURCES_PROD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].prod(axis=1, skipna=True, min_count=1)

df['NEW_EXT_SOURCES_MAD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mad(axis=1, skipna=True)

df['NEW_EXT_SOURCES_GEO'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].apply(geometric_mean, axis=1)

# 特徴量4: 就労期間を年齢で除した値
df['DAYS_EMPLOYED_div_BERTH'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']

# 特徴量5: 年金支払金額を所得金額で除した値
df['ANNUITY_div_INCOME'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']

# 特徴量6: 年金支払額を借入金で除した値
df['ANNUITY_div_CREDIT'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

### 
df['credit_goods_price_ratio'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
 
df['credit_downpayment'] = df['AMT_GOODS_PRICE'] - df['AMT_CREDIT']

df['USER_AGE'] = abs(df['DAYS_BIRTH'])/364.25

df['NEW_CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']

df['NEW_CAR_TO_EMPLOY_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']

df['NEW_PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']

df['NEW_PHONE_TO_EMPLOYED_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']

df['NEW_CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']

df['NEW_INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']

df['NEW_INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']

### 2022/06/25
df['app_EXT_SOURCE_prod'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
df['app_EXT_SOURCE_1_times_EXT_SOURCE_2'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2']
df['app_EXT_SOURCE_1_times_EXT_SOURCE_3'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_3']
df['app_EXT_SOURCE_2_times_EXT_SOURCE_3'] = df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
df['app_EXT_SOURCE_1_times_DAYS_EMPLOYED'] = df['EXT_SOURCE_1'] * df['DAYS_EMPLOYED']
df['app_EXT_SOURCE_2_times_DAYS_EMPLOYED'] = df['EXT_SOURCE_2'] * df['DAYS_EMPLOYED']
df['app_EXT_SOURCE_3_times_DAYS_EMPLOYED'] = df['EXT_SOURCE_3'] * df['DAYS_EMPLOYED']
df['app_EXT_SOURCE_1_PER_DAYS_BIRTH'] = df['EXT_SOURCE_1'] / df['DAYS_BIRTH']
df['app_EXT_SOURCE_2_PER_DAYS_BIRTH'] = df['EXT_SOURCE_2'] / df['DAYS_BIRTH']
df['app_EXT_SOURCE_3_PER_DAYS_BIRTH'] = df['EXT_SOURCE_3'] / df['DAYS_BIRTH']

### 2022/06/26
df['app_most_popular_AMT_GOODS_PRICE'] = df['AMT_GOODS_PRICE'] \
    .isin([225000, 450000, 675000, 900000]).map({True: 1, False: 0})
df['app_popular_AMT_GOODS_PRICE'] = df['AMT_GOODS_PRICE'] \
    .isin([1125000, 1350000, 1575000, 1800000, 2250000]).map({True: 1, False: 0})
df['app_OWN_CAR_AGE_div_DAYS_BIRTH'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
df['app_OWN_CAR_AGE_div_DAYS_EMPLOYED'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
df['app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
df['app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
df['app_DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] - df['DAYS_BIRTH']
df['app_DAYS_EMPLOYED_div_DAYS_BIRTH'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
df['app_CNT_CHILDREN_div_CNT_FAM_MEMBERS'] = df['CNT_CHILDREN'] / df['CNT_FAM_MEMBERS']

### 2022/06/29
#### EXT_SOURCE_X FEATURE
#df['APP_SCORE1_TO_BIRTH_RATIO'] = df['EXT_SOURCE_1'] / (df['DAYS_BIRTH'] / 365.25)
#df['APP_SCORE2_TO_BIRTH_RATIO'] = df['EXT_SOURCE_2'] / (df['DAYS_BIRTH'] / 365.25)
#df['APP_SCORE3_TO_BIRTH_RATIO'] = df['EXT_SOURCE_3'] / (df['DAYS_BIRTH'] / 365.25)
#df['APP_SCORE1_TO_EMPLOY_RATIO'] = df['EXT_SOURCE_1'] / (df['DAYS_EMPLOYED'] / 365.25)
#### Time ratios
#df['ID_TO_BIRTH_RATIO'] = df['DAYS_ID_PUBLISH'] / df['DAYS_BIRTH']

### one-hotencoding
encoding_columns = []

#### one-hotencoding処理対象列を取得
for col in df.columns:
    if df[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

#### one-hotencoding処理対象列を追加
encoding_columns.append('CODE_GENDER')
encoding_columns.append('FLAG_OWN_CAR')
encoding_columns.append('FLAG_OWN_REALTY')
encoding_columns.append('NAME_INCOME_TYPE')
encoding_columns.append('NAME_FAMILY_STATUS')

#### one-hotencoding実行
for col in encoding_columns:
    df = pd.get_dummies(df, columns=[col], dummy_na=True)

### 列名の不正な文字を置換(ドット対応)
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))

### 極大値、極小値をNULLへ置換
df = df.replace([np.inf, -np.inf], np.nan)

len(df.columns)

df.head()

NAME_CONTRACT_TYPE
NAME_TYPE_SUITE
NAME_EDUCATION_TYPE
NAME_HOUSING_TYPE
OCCUPATION_TYPE
WEEKDAY_APPR_PROCESS_START
ORGANIZATION_TYPE
FONDKAPREMONT_MODE
HOUSETYPE_MODE
WALLSMATERIAL_MODE
EMERGENCYSTATE_MODE


,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_INCOME_TYPE_4_0,NAME_INCOME_TYPE_5_0,NAME_INCOME_TYPE_6_0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0_0,NAME_FAMILY_STATUS_1_0,NAME_FAMILY_STATUS_2_0,NAME_FAMILY_STATUS_3_0,NAME_FAMILY_STATUS_4_0,NAME_FAMILY_STATUS_nan
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0,0,0,0,0,1,0,0,0,0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,0,0,0,0,1,0,0,0,0,0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,0,0,0,0,0,1,0,0,0,0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,0,0,0,0,0,0,1,0,0,0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,0,0,0,0,0,1,0,0,0,0


In [7]:
### indexを削除
df=df.reset_index()
df.head()

,level_0,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,NAME_INCOME_TYPE_4_0,NAME_INCOME_TYPE_5_0,NAME_INCOME_TYPE_6_0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0_0,NAME_FAMILY_STATUS_1_0,NAME_FAMILY_STATUS_2_0,NAME_FAMILY_STATUS_3_0,NAME_FAMILY_STATUS_4_0,NAME_FAMILY_STATUS_nan
0,0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,0,0,0,0,0,1,0,0,0,0
1,1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,0,0,0,0,1,0,0,0,0,0
2,2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,0,0,0,0,0,1,0,0,0,0
3,3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,0,0,0,0,0,0,1,0,0,0
4,4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,0,0,0,0,0,1,0,0,0,0


In [8]:
### indexを削除
df = df.drop(columns=['index'])
df.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_INCOME_TYPE_4_0,NAME_INCOME_TYPE_5_0,NAME_INCOME_TYPE_6_0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0_0,NAME_FAMILY_STATUS_1_0,NAME_FAMILY_STATUS_2_0,NAME_FAMILY_STATUS_3_0,NAME_FAMILY_STATUS_4_0,NAME_FAMILY_STATUS_nan
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0,0,0,0,0,1,0,0,0,0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,0,0,0,0,1,0,0,0,0,0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,0,0,0,0,0,1,0,0,0,0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,0,0,0,0,0,0,1,0,0,0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,0,0,0,0,0,1,0,0,0,0


In [9]:
# infをnanへ置換
bureau = bureau.replace([np.inf, -np.inf], np.nan)
bureau_balance = bureau_balance.replace([np.inf, -np.inf], np.nan)
pos_cash_balance = pos_cash_balance.replace([np.inf, -np.inf], np.nan)
credit_card_balance = credit_card_balance.replace([np.inf, -np.inf], np.nan)
previous_application = previous_application.replace([np.inf, -np.inf], np.nan)
installments_payments = installments_payments.replace([np.inf, -np.inf], np.nan)

pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [10]:
baseline = Baseline('SK_ID_CURR')

pos = pos_cash_balance.copy()
pos = baseline.reduce_mem_usage(pos)
print(pos.shape)
pos.head()

Memory usage of dataframe is 610.43 MB
Memory usage of optimization is 290039510.00 MB
Decreased by -47513511.5%
(10001358, 8)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [11]:
pos_ohe = pd.get_dummies(pos, columns=['NAME_CONTRACT_STATUS'], dummy_na=True)
col_ohe = sorted(list(set(pos_ohe.columns) - set(pos.columns)))
print(len(col_ohe))
col_ohe

10


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Amortized debt',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Canceled',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Returned to the store',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_XNA',
 'NAME_CONTRACT_STATUS_nan']

In [12]:
pos_ohe_agg = pos_ohe.groupby('SK_ID_CURR').agg({
    "MONTHS_BALANCE": ["mean", "std", "min", "max"]
    ,"CNT_INSTALMENT": ["mean", "std", "min", "max"]
    ,"CNT_INSTALMENT_FUTURE": ["mean", "std", "min", "max"]
    ,"SK_DPD": ["mean", "std", "min", "max"]
    ,"SK_DPD_DEF": ["mean", "std", "min", "max"]
    # カテゴリ変数をone-hot-encodingした値の集約
    ,"NAME_CONTRACT_STATUS_Active": ["mean"]
    ,"NAME_CONTRACT_STATUS_Amortized debt": ["mean"]
    ,"NAME_CONTRACT_STATUS_Approved": ["mean"]
    ,"NAME_CONTRACT_STATUS_Canceled": ["mean"]
    ,"NAME_CONTRACT_STATUS_Completed": ["mean"]
    ,"NAME_CONTRACT_STATUS_Demand": ["mean"]
    ,"NAME_CONTRACT_STATUS_Returned to the store": ["mean"]
    ,"NAME_CONTRACT_STATUS_Signed": ["mean"]
    ,"NAME_CONTRACT_STATUS_XNA": ["mean"]
    ,"NAME_CONTRACT_STATUS_nan": ["mean"]
    # IDのユニーク数をカウント
    ,"SK_ID_PREV":["count", "nunique"]
})

# カラム名の付与
pos_ohe_agg.columns = [i + "_" + j for i,j in pos_ohe_agg.columns]
pos_ohe_agg = pos_ohe_agg.reset_index(drop=False)

print(pos_ohe_agg.shape)
pos_ohe_agg.head()

(337252, 33)


,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,CNT_INSTALMENT_mean,CNT_INSTALMENT_std,CNT_INSTALMENT_min,CNT_INSTALMENT_max,CNT_INSTALMENT_FUTURE_mean,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,-72.555556,20.863312,-96,-53,4.000000,0.000000,4.0,4.0,1.444444,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9,2
1,100002,-10.000000,5.627314,-19,-1,24.000000,0.000000,24.0,24.0,15.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,19,1
2,100003,-43.785714,24.640162,-77,-18,10.107142,2.806597,6.0,12.0,5.785714,...,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,28,3
3,100004,-25.500000,1.290994,-27,-24,3.750000,0.500000,3.0,4.0,2.250000,...,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,4,1
4,100005,-20.000000,3.316625,-25,-15,11.700000,0.948683,9.0,12.0,7.200000,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11,1


In [13]:
### infをnanへ置換
pos_ohe_agg = pos_ohe_agg.replace([np.inf, -np.inf], np.nan)
pos_ohe_agg.head()

,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,CNT_INSTALMENT_mean,CNT_INSTALMENT_std,CNT_INSTALMENT_min,CNT_INSTALMENT_max,CNT_INSTALMENT_FUTURE_mean,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,-72.555556,20.863312,-96,-53,4.000000,0.000000,4.0,4.0,1.444444,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9,2
1,100002,-10.000000,5.627314,-19,-1,24.000000,0.000000,24.0,24.0,15.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,19,1
2,100003,-43.785714,24.640162,-77,-18,10.107142,2.806597,6.0,12.0,5.785714,...,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,28,3
3,100004,-25.500000,1.290994,-27,-24,3.750000,0.500000,3.0,4.0,2.250000,...,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,4,1
4,100005,-20.000000,3.316625,-25,-15,11.700000,0.948683,9.0,12.0,7.200000,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11,1


In [14]:
credit = credit_card_balance_origin.copy()
cerdit = baseline.reduce_mem_usage(credit)
print(credit.shape)
credit.head()

Memory usage of dataframe is 673.88 MB
Memory usage of optimization is 334107272.00 MB
Decreased by -49579318.7%
(3840312, 23)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,...,0.000000,0.000000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.554688,45000,2250.0,2250.0,0.0,0.0,2250.000000,...,64875.554688,64875.554688,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.224609,450000,0.0,0.0,0.0,0.0,2250.000000,...,31460.085938,31460.085938,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.109375,225000,2250.0,2250.0,0.0,0.0,11795.759766,...,233048.968750,233048.968750,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.468750,450000,0.0,11547.0,0.0,11547.0,22924.890625,...,453919.468750,453919.468750,0.0,1,0.0,1.0,101.0,Active,0,0


In [15]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in credit.columns:
    if credit[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

encoding_columns

NAME_CONTRACT_STATUS


['NAME_CONTRACT_STATUS']

In [16]:
# one-hot-encoding
for col in encoding_columns:
    credit_ohe = pd.get_dummies(credit, columns=[col], dummy_na=True)
    col_ohe = sorted(list(set(credit_ohe.columns) - set(credit.columns)))

print(len(col_ohe))
col_ohe

8


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Refused',
 'NAME_CONTRACT_STATUS_Sent proposal',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_nan']

In [17]:
credit_ohe_agg = credit_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'SK_ID_CURR': ['mean', 'std', 'min', 'max']
        ,'MONTHS_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_INST_MIN_REGULARITY': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_TOTAL_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_RECEIVABLE_PRINCIPAL': ['mean', 'std', 'min', 'max']
        ,'AMT_RECIVABLE': ['mean', 'std', 'min', 'max']
        ,'AMT_TOTAL_RECEIVABLE': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_INSTALMENT_MATURE_CUM': ['mean', 'std', 'min', 'max']
        ,'SK_DPD': ['mean', 'std', 'min', 'max']
        ,'SK_DPD_DEF': ['mean', 'std', 'min', 'max']
        # 
        ,'NAME_CONTRACT_STATUS_Active': ['mean']
        ,'NAME_CONTRACT_STATUS_Approved': ['mean']
        ,'NAME_CONTRACT_STATUS_Completed': ['mean']
        ,'NAME_CONTRACT_STATUS_Demand': ['mean']
        ,'NAME_CONTRACT_STATUS_Refused': ['mean']
        ,'NAME_CONTRACT_STATUS_Sent proposal': ['mean']
        ,'NAME_CONTRACT_STATUS_Signed': ['mean']
        ,'NAME_CONTRACT_STATUS_nan': ['mean']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV":["count", "nunique"]
    }
)

# カラム名の付与
credit_ohe_agg.columns = [i + "_" + j for i,j in credit_ohe_agg.columns]
credit_ohe_agg = credit_ohe_agg.reset_index(drop=False)

print(credit_ohe_agg.shape)
credit_ohe_agg.head()

(103558, 95)


,SK_ID_CURR,SK_ID_CURR_mean,SK_ID_CURR_std,SK_ID_CURR_min,SK_ID_CURR_max,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,AMT_BALANCE_mean,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100006,100006.0,0.0,100006,100006,-3.5,1.870829,-6,-1,0.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,6,1
1,100011,100011.0,0.0,100011,100011,-38.5,21.505813,-75,-2,54482.109375,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,74,1
2,100013,100013.0,0.0,100013,100013,-48.5,27.856777,-96,-1,18159.919922,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,96,1
3,100021,100021.0,0.0,100021,100021,-10.0,5.049752,-18,-2,0.000000,...,0.411765,0.0,0.588235,0.0,0.0,0.0,0.0,0.0,17,1
4,100023,100023.0,0.0,100023,100023,-7.5,2.449490,-11,-4,0.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,8,1


In [18]:
# infをnanへ置換
credit_ohe_agg = credit_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [19]:
previous = previous_application_origin.copy()
previous = baseline.reduce_mem_usage(credit)
print(previous.shape)
previous.head()

Memory usage of dataframe is 318.63 MB
Memory usage of optimization is 334107272.00 MB
Decreased by -104857500.0%
(3840312, 23)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,...,0.000000,0.000000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.554688,45000,2250.0,2250.0,0.0,0.0,2250.000000,...,64875.554688,64875.554688,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.224609,450000,0.0,0.0,0.0,0.0,2250.000000,...,31460.085938,31460.085938,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.109375,225000,2250.0,2250.0,0.0,0.0,11795.759766,...,233048.968750,233048.968750,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.468750,450000,0.0,11547.0,0.0,11547.0,22924.890625,...,453919.468750,453919.468750,0.0,1,0.0,1.0,101.0,Active,0,0


In [20]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in previous.columns:
    if previous[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

#encoding_columns

NAME_CONTRACT_STATUS


In [21]:
# one-hot-encoding
for col in encoding_columns:
    previous_ohe = pd.get_dummies(previous, columns=[col], dummy_na=True)
    col_ohe = sorted(list(set(previous_ohe.columns) - set(previous.columns)))

print(len(col_ohe))
col_ohe

8


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Refused',
 'NAME_CONTRACT_STATUS_Sent proposal',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_nan']

In [22]:
previous_ohe_agg = credit_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'MONTHS_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_INST_MIN_REGULARITY': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_TOTAL_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_RECEIVABLE_PRINCIPAL': ['mean', 'std', 'min', 'max']
        ,'AMT_RECIVABLE': ['mean', 'std', 'min', 'max']
        ,'AMT_TOTAL_RECEIVABLE': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_INSTALMENT_MATURE_CUM': ['mean', 'std', 'min', 'max']
        ,'SK_DPD': ['mean', 'std', 'min', 'max']
        ,'SK_DPD_DEF': ['mean', 'std', 'min', 'max']
        # 
        ,'NAME_CONTRACT_STATUS_Active': ['mean']
        ,'NAME_CONTRACT_STATUS_Approved': ['mean']
        ,'NAME_CONTRACT_STATUS_Completed': ['mean']
        ,'NAME_CONTRACT_STATUS_Demand': ['mean']
        ,'NAME_CONTRACT_STATUS_Refused': ['mean']
        ,'NAME_CONTRACT_STATUS_Sent proposal': ['mean']
        ,'NAME_CONTRACT_STATUS_Signed': ['mean']
        ,'NAME_CONTRACT_STATUS_nan': ['mean']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV":["count", "nunique"]
    }
)

# カラム名の付与
previous_ohe_agg.columns = [i + "_" + j for i,j in previous_ohe_agg.columns]
previous_ohe_agg = previous_ohe_agg.reset_index(drop=False)

print(previous_ohe_agg.shape)
previous_ohe_agg.head()

(103558, 91)


,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,AMT_BALANCE_mean,AMT_BALANCE_std,AMT_BALANCE_min,AMT_BALANCE_max,AMT_CREDIT_LIMIT_ACTUAL_mean,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100006,-3.5,1.870829,-6,-1,0.000000,0.000000,0.0,0.00000,270000.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,6,1
1,100011,-38.5,21.505813,-75,-2,54482.109375,68127.238133,0.0,189000.00000,164189.189189,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,74,1
2,100013,-48.5,27.856777,-96,-1,18159.919922,43237.406465,0.0,161420.21875,131718.750000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,96,1
3,100021,-10.0,5.049752,-18,-2,0.000000,0.000000,0.0,0.00000,675000.000000,...,0.411765,0.0,0.588235,0.0,0.0,0.0,0.0,0.0,17,1
4,100023,-7.5,2.449490,-11,-4,0.000000,0.000000,0.0,0.00000,135000.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,8,1


In [23]:
# infをnanへ置換
previous_ohe_agg = previous_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [24]:
installments = installments_payments_origin.copy()
installments = baseline.reduce_mem_usage(installments)
print(installments.shape)
installments.head()

Memory usage of dataframe is 830.41 MB
Memory usage of optimization is 408162158.00 MB
Decreased by -49151908.2%
(13605401, 8)


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.359863,6948.359863
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525024,1716.525024
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000000,25425.000000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130859,24350.130859
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040039,2160.584961


In [25]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in installments.columns:
    if installments[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

encoding_columns

[]

In [26]:
# one-hot-encoding
#for col in encoding_columns:
#    installments_ohe = pd.get_dummies(installments, columns=[col], dummy_na=True)
#    col_ohe = sorted(list(set(installments_ohe.columns) - set(installments.columns)))

#print(len(col_ohe))
#col_ohe
installments_ohe = installments.copy()

In [27]:
installments_ohe = installments_ohe.rename(columns={'SK_ID_PREV': 'SK_ID_PREV_IP'})

In [28]:
installments_ohe_agg = installments_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'NUM_INSTALMENT_VERSION': ['mean', 'std', 'min', 'max']
        ,'NUM_INSTALMENT_NUMBER': ['mean', 'std', 'min', 'max']
        ,'DAYS_INSTALMENT': ['mean', 'std', 'min', 'max']
        ,'DAYS_ENTRY_PAYMENT': ['mean', 'std', 'min', 'max']
        ,'AMT_INSTALMENT': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT': ['mean', 'std', 'min', 'max']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV_IP":["count", "nunique"]
    }
)

# カラム名の付与
installments_ohe_agg.columns = [i + "_" + j for i,j in installments_ohe_agg.columns]
installments_ohe_agg = installments_ohe_agg.reset_index(drop=False)

print(installments_ohe_agg.shape)
installments_ohe_agg.head()

(339587, 27)


,SK_ID_CURR,NUM_INSTALMENT_VERSION_mean,NUM_INSTALMENT_VERSION_std,NUM_INSTALMENT_VERSION_min,NUM_INSTALMENT_VERSION_max,NUM_INSTALMENT_NUMBER_mean,NUM_INSTALMENT_NUMBER_std,NUM_INSTALMENT_NUMBER_min,NUM_INSTALMENT_NUMBER_max,DAYS_INSTALMENT_mean,...,AMT_INSTALMENT_mean,AMT_INSTALMENT_std,AMT_INSTALMENT_min,AMT_INSTALMENT_max,AMT_PAYMENT_mean,AMT_PAYMENT_std,AMT_PAYMENT_min,AMT_PAYMENT_max,SK_ID_PREV_IP_count,SK_ID_PREV_IP_nunique
0,100001,1.142857,0.377964,1.0,2.0,2.714286,1.112697,1,4,-2187.714355,...,5885.132324,5076.676763,3951.000000,17397.900391,5885.132324,5076.676763,3951.000000,17397.900391,7,2
1,100002,1.052632,0.229416,1.0,2.0,10.000000,5.627314,1,19,-295.000000,...,11559.247070,10058.037883,9251.775391,53093.746094,11559.247070,10058.037883,9251.775391,53093.746094,19,1
2,100003,1.040000,0.200000,1.0,2.0,5.080000,3.134751,1,12,-1378.160034,...,64754.585938,110542.594872,6662.970215,560835.375000,64754.585938,110542.594872,6662.970215,560835.375000,25,3
3,100004,1.333333,0.577350,1.0,2.0,2.000000,1.000000,1,3,-754.000000,...,7096.154785,3011.871719,5357.250000,10573.964844,7096.154785,3011.871719,5357.250000,10573.964844,3,1
4,100005,1.111111,0.333333,1.0,2.0,5.000000,2.738613,1,9,-586.000000,...,6240.205078,4281.014648,4813.200195,17656.244141,6240.205078,4281.014648,4813.200195,17656.244141,9,1


In [29]:
# infをnanへ置換
installments_ohe_agg = installments_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [30]:
bureau = bureau_origin.copy()
bureau = baseline.reduce_mem_usage(bureau)
print(bureau.shape)
bureau.head()

Memory usage of dataframe is 222.62 MB
Memory usage of optimization is 125299372.00 MB
Decreased by -56283785.5%
(1716428, 17)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [31]:
### 
for col in bureau.columns:
    bureau[col] = bureau[col].replace('\s', '_', regex=True)
    bureau[col] = bureau[col].replace(':', '_', regex=True)
    bureau[col] = bureau[col].replace('/', '_', regex=True)
    
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency_1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer_credit,-131,NaN
1,215354,5714463,Active,currency_1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit_card,-20,NaN
2,215354,5714464,Active,currency_1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer_credit,-16,NaN
3,215354,5714465,Active,currency_1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit_card,-16,NaN
4,215354,5714466,Active,currency_1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer_credit,-21,NaN


In [32]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in bureau.columns:
    if bureau[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

CREDIT_ACTIVE
CREDIT_CURRENCY
CREDIT_TYPE


In [33]:
# one-hot-encoding
bureau_ohe = pd.get_dummies(bureau, columns=['CREDIT_ACTIVE'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

bureau_ohe = pd.get_dummies(bureau_ohe, columns=['CREDIT_CURRENCY'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

bureau_ohe = pd.get_dummies(bureau_ohe, columns=['CREDIT_TYPE'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

print(len(col_ohe))
col_ohe

26


['CREDIT_ACTIVE_Active',
 'CREDIT_ACTIVE_Bad_debt',
 'CREDIT_ACTIVE_Closed',
 'CREDIT_ACTIVE_Sold',
 'CREDIT_ACTIVE_nan',
 'CREDIT_CURRENCY_currency_1',
 'CREDIT_CURRENCY_currency_2',
 'CREDIT_CURRENCY_currency_3',
 'CREDIT_CURRENCY_currency_4',
 'CREDIT_CURRENCY_nan',
 'CREDIT_TYPE_Another_type_of_loan',
 'CREDIT_TYPE_Car_loan',
 'CREDIT_TYPE_Cash_loan_(non-earmarked)',
 'CREDIT_TYPE_Consumer_credit',
 'CREDIT_TYPE_Credit_card',
 'CREDIT_TYPE_Interbank_credit',
 'CREDIT_TYPE_Loan_for_business_development',
 'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)',
 'CREDIT_TYPE_Loan_for_the_purchase_of_equipment',
 'CREDIT_TYPE_Loan_for_working_capital_replenishment',
 'CREDIT_TYPE_Microloan',
 'CREDIT_TYPE_Mobile_operator_loan',
 'CREDIT_TYPE_Mortgage',
 'CREDIT_TYPE_Real_estate_loan',
 'CREDIT_TYPE_Unknown_type_of_loan',
 'CREDIT_TYPE_nan']

In [34]:
bureau_ohe = bureau_ohe.rename(columns={'SK_ID_BUREAU': 'SK_ID_BUREAU_BUREAU'})

In [35]:
bureau_ohe_agg = bureau_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'DAYS_CREDIT': ['mean', 'std', 'min', 'max', 'var']
        ,'CREDIT_DAY_OVERDUE': ['mean', 'std', 'min', 'max']
        ,'DAYS_CREDIT_ENDDATE': ['mean', 'std', 'min', 'max']
        ,'DAYS_ENDDATE_FACT': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_MAX_OVERDUE': ['mean', 'std', 'min', 'max']
        ,'CNT_CREDIT_PROLONG': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_SUM': ['mean', 'std', 'min', 'max', 'sum']
        ,'AMT_CREDIT_SUM_DEBT': ['mean', 'std', 'min', 'max', 'sum']
        ,'AMT_CREDIT_SUM_LIMIT': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_SUM_OVERDUE': ['mean', 'std', 'min', 'max']
        ,'DAYS_CREDIT_UPDATE': ['mean', 'std', 'min', 'max']
        ,'AMT_ANNUITY': ['mean', 'std', 'min', 'max']
        # 
        ,'CREDIT_TYPE_Another_type_of_loan': ['mean']
        ,'CREDIT_TYPE_Car_loan': ['mean']
        ,'CREDIT_TYPE_Cash_loan_(non-earmarked)': ['mean']
        ,'CREDIT_TYPE_Consumer_credit': ['mean']
        ,'CREDIT_TYPE_Credit_card': ['mean']
        ,'CREDIT_TYPE_Interbank_credit': ['mean']
        ,'CREDIT_TYPE_Loan_for_business_development': ['mean']
        ,'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)': ['mean']
        ,'CREDIT_TYPE_Loan_for_the_purchase_of_equipment': ['mean']
        ,'CREDIT_TYPE_Loan_for_working_capital_replenishment': ['mean']
        ,'CREDIT_TYPE_Microloan': ['mean']
        ,'CREDIT_TYPE_Mobile_operator_loan': ['mean']
        ,'CREDIT_TYPE_Mortgage': ['mean']
        ,'CREDIT_TYPE_Real_estate_loan': ['mean']
        ,'CREDIT_TYPE_Unknown_type_of_loan': ['mean']
        ,'CREDIT_TYPE_nan': ['mean']
        ,'CREDIT_ACTIVE_Active': ['mean']
        ,'CREDIT_ACTIVE_Bad_debt': ['mean']
        ,'CREDIT_ACTIVE_Closed': ['mean']
        ,'CREDIT_ACTIVE_Sold': ['mean']
        ,'CREDIT_ACTIVE_nan': ['mean']
        ,'CREDIT_CURRENCY_currency_1': ['mean']
        ,'CREDIT_CURRENCY_currency_2': ['mean']
        ,'CREDIT_CURRENCY_currency_3': ['mean']
        ,'CREDIT_CURRENCY_currency_4': ['mean']
        ,'CREDIT_CURRENCY_nan': ['mean']
        ,'CREDIT_TYPE_Another_type_of_loan': ['mean']
        ,'CREDIT_TYPE_Car_loan': ['mean']
        ,'CREDIT_TYPE_Cash_loan_(non-earmarked)': ['mean']
        ,'CREDIT_TYPE_Consumer_credit': ['mean']
        ,'CREDIT_TYPE_Credit_card': ['mean']
        ,'CREDIT_TYPE_Interbank_credit': ['mean']
        ,'CREDIT_TYPE_Loan_for_business_development': ['mean']
        ,'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)': ['mean']
        ,'CREDIT_TYPE_Loan_for_the_purchase_of_equipment': ['mean']
        ,'CREDIT_TYPE_Loan_for_working_capital_replenishment': ['mean']
        ,'CREDIT_TYPE_Microloan': ['mean']
        ,'CREDIT_TYPE_Mobile_operator_loan': ['mean']
        ,'CREDIT_TYPE_Mortgage': ['mean']
        ,'CREDIT_TYPE_Real_estate_loan': ['mean']
        ,'CREDIT_TYPE_Unknown_type_of_loan': ['mean']
        ,'CREDIT_TYPE_nan': ['mean']
        # IDのユニーク数をカウント
        ,'SK_ID_BUREAU_BUREAU': ["count", "nunique"]
    }
)

# カラム名の付与
bureau_ohe_agg.columns = [i + "_" + j for i,j in bureau_ohe_agg.columns]
bureau_ohe_agg = bureau_ohe_agg.reset_index(drop=False)

#
bureau_ohe_agg['debt_credit_ratio_None'] = bureau_ohe_agg['AMT_CREDIT_SUM_DEBT_sum'] / bureau_ohe_agg['AMT_CREDIT_SUM_sum']

print(bureau_ohe_agg.shape)
bureau_ohe_agg.head()

(305811, 81)


,SK_ID_CURR,DAYS_CREDIT_mean,DAYS_CREDIT_std,DAYS_CREDIT_min,DAYS_CREDIT_max,DAYS_CREDIT_var,CREDIT_DAY_OVERDUE_mean,CREDIT_DAY_OVERDUE_std,CREDIT_DAY_OVERDUE_min,CREDIT_DAY_OVERDUE_max,...,CREDIT_ACTIVE_Sold_mean,CREDIT_ACTIVE_nan_mean,CREDIT_CURRENCY_currency_1_mean,CREDIT_CURRENCY_currency_2_mean,CREDIT_CURRENCY_currency_3_mean,CREDIT_CURRENCY_currency_4_mean,CREDIT_CURRENCY_nan_mean,SK_ID_BUREAU_BUREAU_count,SK_ID_BUREAU_BUREAU_nunique,debt_credit_ratio_None
0,100001,-735.000000,489.942514,-1572,-49,240043.666667,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7,7,0.410555
1,100002,-874.000000,431.451040,-1437,-103,186150.000000,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8,8,0.284122
2,100003,-1400.750000,909.826128,-2586,-606,827783.583333,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4,4,0.000000
3,100004,-867.000000,649.124025,-1326,-408,421362.000000,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2,2,0.000000
4,100005,-190.666667,162.297053,-373,-62,26340.333333,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3,3,0.864992


In [36]:
# consumer_annuity_to_max_installment_ratio
app_train_mean1 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_consumer_annuity_to_max_installment_ratio.csv')
app_test_mean1 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_consumer_annuity_to_max_installment_ratio.csv')
app_train_mean1 = app_train_mean1.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean1 = app_test_mean1.rename(columns={'sk_id_curr': 'SK_ID_CURR'})

### installment_payment_ratio_1000_mean_mean
#### consumer_installment_payment_ratio_1000_mean_mean
app_train_mean2 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_consumer_installment_payment_ratio_1000_mean_mean.csv')
app_test_mean2 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_consumer_installment_payment_ratio_1000_mean_mean.csv')
app_train_mean2 = app_train_mean2.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean2 = app_test_mean2.rename(columns={'sk_id_curr': 'SK_ID_CURR'})


#### revo_installment_payment_ratio_1000_mean_mean
app_train_mean3 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_revo_installment_payment_ratio_1000_mean_mean.csv')
app_test_mean3 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_revo_installment_payment_ratio_1000_mean_mean.csv')
app_train_mean3 = app_train_mean3.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean3 = app_test_mean3.rename(columns={'sk_id_curr': 'SK_ID_CURR'})

#### installment_payment_ratio_1000_mean_mean
app_train_mean4 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
app_test_mean4 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
app_train_mean4 = app_train_mean4.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean4 = app_test_mean4.rename(columns={'sk_id_curr': 'SK_ID_CURR'})


In [37]:
# infをnanへ置換
app_train_mean1 = app_train_mean1.replace([np.inf, -np.inf], np.nan)
app_test_mean1 = app_train_mean1.replace([np.inf, -np.inf], np.nan)

app_train_mean2 = app_train_mean2.replace([np.inf, -np.inf], np.nan)
app_test_mean2 = app_test_mean2.replace([np.inf, -np.inf], np.nan)

app_train_mean3 = app_train_mean3.replace([np.inf, -np.inf], np.nan)
app_test_mean3 = app_test_mean3.replace([np.inf, -np.inf], np.nan)

app_train_mean4 = app_train_mean4.replace([np.inf, -np.inf], np.nan)
app_test_mean4 = app_train_mean4.replace([np.inf, -np.inf], np.nan)

In [38]:
### bureau_balance
df = df.merge(bureau_ohe_agg, how='left', on='SK_ID_CURR')
df.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,CREDIT_ACTIVE_Sold_mean,CREDIT_ACTIVE_nan_mean,CREDIT_CURRENCY_currency_1_mean,CREDIT_CURRENCY_currency_2_mean,CREDIT_CURRENCY_currency_3_mean,CREDIT_CURRENCY_currency_4_mean,CREDIT_CURRENCY_nan_mean,SK_ID_BUREAU_BUREAU_count,SK_ID_BUREAU_BUREAU_nunique,debt_credit_ratio_None
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.0,8.0,0.284122
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,4.0,0.000000
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,0.000000
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.000000


In [39]:
### previos_application
df = df.merge(previous_ohe_agg, how='left', on='SK_ID_CURR')
df.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
### pos_cash
df = df.merge(pos_ohe_agg, how='left', on='SK_ID_CURR')
df.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [41]:
### instllment_payment
df = df.merge(installments_ohe_agg, how='left', on='SK_ID_CURR')
df.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,AMT_INSTALMENT_mean,AMT_INSTALMENT_std,AMT_INSTALMENT_min,AMT_INSTALMENT_max,AMT_PAYMENT_mean,AMT_PAYMENT_std,AMT_PAYMENT_min,AMT_PAYMENT_max,SK_ID_PREV_IP_count,SK_ID_PREV_IP_nunique
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,11559.247070,10058.037883,9251.775391,53093.746094,11559.247070,10058.037883,9251.775391,53093.746094,19.0,1.0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,64754.585938,110542.594872,6662.970215,560835.375000,64754.585938,110542.594872,6662.970215,560835.375000,25.0,3.0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,7096.154785,3011.871719,5357.250000,10573.964844,7096.154785,3011.871719,5357.250000,10573.964844,3.0,1.0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,62947.085938,168097.620686,2482.919922,691786.875000,62947.085938,168097.620686,2482.919922,691786.875000,16.0,3.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,12666.444336,7852.910602,1821.780029,22678.785156,12214.060547,8048.060783,0.180000,22678.785156,66.0,5.0


In [42]:
### credit_card_balance
df = df.merge(credit_ohe_agg, how='left', on='SK_ID_CURR')
df.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean_y,NAME_CONTRACT_STATUS_Sent proposal_mean_y,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
### infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)

In [44]:
### 変数選択001
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 636 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 619 features are remained after removing non-informative features
119 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 500 features are remained after removing features not interesting for LightGBM classifier
---=> 36 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 536 features

---=> df final shape: (356250, 536)  <=--- 



,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_high_risk,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_low_risk,CNT_CREDIT_PROLONG_max_high_risk,CNT_CREDIT_PROLONG_max_low_risk,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,NUM_INSTALMENT_VERSION_min_low_risk
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0,1,0,1,0,0,0,0,0,1
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0,1,0,1,0,0,0,0,0,1
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0,1,0,1,0,0,0,0,0,1
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,0,1,0,0,1,0,1,0,0,1
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0,1,0,1,0,0,0,0,0,1


In [45]:
### 変数選択002
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 536 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 535 features are remained after removing non-informative features
41 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 494 features are remained after removing features not interesting for LightGBM classifier
---=> 13 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 507 features

---=> df final shape: (356250, 507)  <=--- 



,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,AMT_REQ_CREDIT_BUREAU_HOUR_high_risk,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_DAY_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_low_risk,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_low_risk,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0,1,1,1,1,0,0,0,0,0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0,1,1,1,1,0,0,0,0,0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0,1,1,1,1,0,0,0,0,0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,0,0,0,0,1,1,0,1,0,0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0,1,1,1,1,0,0,0,0,0


In [46]:
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,AMT_REQ_CREDIT_BUREAU_HOUR_high_risk,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_DAY_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_low_risk,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_low_risk,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0,1,1,1,1,0,0,0,0,0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0,1,1,1,1,0,0,0,0,0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0,1,1,1,1,0,0,0,0,0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,0,0,0,0,1,1,0,1,0,0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0,1,1,1,1,0,0,0,0,0


In [47]:
# split train, and test datasets
df_train = df[df['TARGET'].notnull()]
df_test = df[df['TARGET'].isnull()]

print(df_test.shape)
df_test.head()

(48744, 507)


,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,AMT_REQ_CREDIT_BUREAU_HOUR_high_risk,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_DAY_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_low_risk,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_low_risk,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk
307506,307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,0,1,1,1,1,0,0,0,0,0
307507,307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,0,1,1,1,1,0,0,0,0,0
307508,307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,0,1,1,1,1,1,0,1,0,1
307509,307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,0,1,1,1,0,1,0,1,0,1
307510,307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0,0,0,0,0,0,0


In [48]:
pred_train_set1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_set1a.csv')
pred_test_set1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_set1a.csv')
pred_test_set1a.head()

,SK_ID_CURR,set1
0,100001,0.040068
1,100005,0.095240
2,100013,0.044435
3,100028,0.036586
4,100038,0.141754


In [49]:
df_train = df_train.merge(pred_train_set1a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_set1a, how='left', on='SK_ID_CURR')
df_test.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,AMT_REQ_CREDIT_BUREAU_HOUR_low_risk,AMT_REQ_CREDIT_BUREAU_DAY_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_low_risk,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_low_risk,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1
0,307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,1,1,1,1,0,0,0,0,0,0.040068
1,307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,1,1,1,1,0,0,0,0,0,0.095240
2,307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,1,1,1,1,1,0,1,0,1,0.044435
3,307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,1,1,1,0,1,0,1,0,1,0.036586
4,307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0,0,0,0,0,0,0.141754


In [50]:
train_set1xgb = pd.read_csv('./home-credit-default-risk/exports/train_set1xgb.csv')
test_set1xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set1xgb.csv')
test_set1xgb.head()

,SK_ID_CURR,set1x
0,100001,0.035234
1,100005,0.107426
2,100013,0.048029
3,100028,0.032975
4,100038,0.161962


In [51]:
df_train = df_train.merge(train_set1xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set1xgb, how='left', on='SK_ID_CURR')
df_test.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,AMT_REQ_CREDIT_BUREAU_DAY_low_risk,AMT_REQ_CREDIT_BUREAU_WEEK_low_risk,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_low_risk,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x
0,307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,1,1,1,0,0,0,0,0,0.040068,0.035234
1,307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,1,1,1,0,0,0,0,0,0.095240,0.107426
2,307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,1,1,1,1,0,1,0,1,0.044435,0.048029
3,307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,1,1,0,1,0,1,0,1,0.036586,0.032975
4,307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0,0,0,0,0,0.141754,0.161962


In [52]:
train_set2a = pd.read_csv('./home-credit-default-risk/exports/train_set2a.csv')
test_set2a =  pd.read_csv('./home-credit-default-risk/exports/test_set2a.csv')
test_set2a.head()

,SK_ID_CURR,set2a
0,100001,0.042769
1,100005,0.103694
2,100013,0.036006
3,100028,0.048812
4,100038,0.173045


In [53]:
df_train = df_train.merge(train_set2a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2a, how='left', on='SK_ID_CURR')
df_test.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,AMT_REQ_CREDIT_BUREAU_WEEK_low_risk,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_low_risk,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a
0,307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,1,1,0,0,0,0,0,0.040068,0.035234,0.042769
1,307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,1,1,0,0,0,0,0,0.095240,0.107426,0.103694
2,307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,1,1,1,0,1,0,1,0.044435,0.048029,0.036006
3,307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,1,0,1,0,1,0,1,0.036586,0.032975,0.048812
4,307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0,0,0,0,0.141754,0.161962,0.173045


In [54]:
train_set2xgb = pd.read_csv('./home-credit-default-risk/exports/train_set2xgb.csv')
test_set2xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set2xgb.csv')
test_set1xgb.head()

,SK_ID_CURR,set1x
0,100001,0.035234
1,100005,0.107426
2,100013,0.048029
3,100028,0.032975
4,100038,0.161962


In [55]:
df_train = df_train.merge(train_set2xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2xgb, how='left', on='SK_ID_CURR')
df_test.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS_low_risk,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x
0,307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,1,0,0,0,0,0,0.040068,0.035234,0.042769,0.035013
1,307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,1,0,0,0,0,0,0.095240,0.107426,0.103694,0.103532
2,307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,1,1,0,1,0,1,0.044435,0.048029,0.036006,0.048415
3,307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,0,1,0,1,0,1,0.036586,0.032975,0.048812,0.032249
4,307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0,0,0,0.141754,0.161962,0.173045,0.150442


In [56]:
train_set2nn = pd.read_csv('./home-credit-default-risk/exports/train_set2nn.csv')
test_set2nn =  pd.read_csv('./home-credit-default-risk/exports/test_set2nn.csv')
test_set2nn.head()

,SK_ID_CURR,set2nn
0,100001,0.048972
1,100005,0.141351
2,100013,0.028088
3,100028,0.039056
4,100038,0.192637


In [57]:
df_train = df_train.merge(train_set2nn, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2nn, how='left', on='SK_ID_CURR')
df_test.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,MONTHS_BALANCE_max_x_high_risk,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn
0,307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,0,0,0,0,0,0.040068,0.035234,0.042769,0.035013,0.048972
1,307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,0,0,0,0,0,0.095240,0.107426,0.103694,0.103532,0.141351
2,307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,1,0,1,0,1,0.044435,0.048029,0.036006,0.048415,0.028088
3,307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,1,0,1,0,1,0.036586,0.032975,0.048812,0.032249,0.039056
4,307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0,0,0.141754,0.161962,0.173045,0.150442,0.192637


In [58]:
train_set3a = pd.read_csv('./home-credit-default-risk/exports/train_set3a.csv')
test_set3a =  pd.read_csv('./home-credit-default-risk/exports/test_set3a.csv')
test_set3a.head()

,SK_ID_CURR,set3a
0,100001,0.049120
1,100005,0.115729
2,100013,0.064166
3,100028,0.038931
4,100038,0.195575


In [59]:
df_train = df_train.merge(train_set3a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set3a, how='left', on='SK_ID_CURR')
df_test.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,MONTHS_BALANCE_max_x_low_risk,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a
0,307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,0,0,0,0,0.040068,0.035234,0.042769,0.035013,0.048972,0.049120
1,307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,0,0,0,0,0.095240,0.107426,0.103694,0.103532,0.141351,0.115729
2,307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,0,1,0,1,0.044435,0.048029,0.036006,0.048415,0.028088,0.064166
3,307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,0,1,0,1,0.036586,0.032975,0.048812,0.032249,0.039056,0.038931
4,307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0,0.141754,0.161962,0.173045,0.150442,0.192637,0.195575


In [60]:
train_set3xgb = pd.read_csv('./home-credit-default-risk/exports/train_set3xgb.csv')
test_set3xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set3xgb.csv')
test_set3xgb.head()

,SK_ID_CURR,set3x
0,100001,0.034310
1,100005,0.127556
2,100013,0.044872
3,100028,0.030915
4,100038,0.201295


In [61]:
df_train = df_train.merge(train_set3xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set3xgb, how='left', on='SK_ID_CURR')
df_test.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,0,0,0,0.040068,0.035234,0.042769,0.035013,0.048972,0.049120,0.034310
1,307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,0,0,0,0.095240,0.107426,0.103694,0.103532,0.141351,0.115729,0.127556
2,307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,1,0,1,0.044435,0.048029,0.036006,0.048415,0.028088,0.064166,0.044872
3,307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,1,0,1,0.036586,0.032975,0.048812,0.032249,0.039056,0.038931,0.030915
4,307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0.141754,0.161962,0.173045,0.150442,0.192637,0.195575,0.201295


In [62]:
### infをnanへ置換
df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0,0,0,0.260928,0.199398,0.242499,0.198794,0.240473,0.223662,0.259405
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0,0,0,0.012171,0.012061,0.017857,0.012648,0.014736,0.024137,0.023080
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0,0,0,0.039950,0.047440,0.033203,0.033060,0.029793,0.035314,0.033290
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,1,0,0,0.030776,0.030071,0.048900,0.025442,0.031804,0.031135,0.022860
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0,0,0,0.035088,0.023555,0.043556,0.039186,0.060946,0.035167,0.033131


In [63]:
### infをnanへ置換
df_test = df_test.replace([np.inf, -np.inf], np.nan)
df_test.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,307511,100001,NaN,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,...,0,0,0,0.040068,0.035234,0.042769,0.035013,0.048972,0.049120,0.034310
1,307512,100005,NaN,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,...,0,0,0,0.095240,0.107426,0.103694,0.103532,0.141351,0.115729,0.127556
2,307513,100013,NaN,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,...,1,0,1,0.044435,0.048029,0.036006,0.048415,0.028088,0.064166,0.044872
3,307514,100028,NaN,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,...,1,0,1,0.036586,0.032975,0.048812,0.032249,0.039056,0.038931,0.030915
4,307515,100038,NaN,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,...,0,0,0,0.141754,0.161962,0.173045,0.150442,0.192637,0.195575,0.201295


In [64]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [65]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a,set3x
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,0,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,0,0,0.260928,0.199398,0.242499,0.198794,0.240473,0.223662,0.259405
100003,1,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,0,0,0,0.012171,0.012061,0.017857,0.012648,0.014736,0.024137,0.023080
100004,2,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,0,0.039950,0.047440,0.033203,0.033060,0.029793,0.035314,0.033290
100006,3,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,1,0,0,0.030776,0.030071,0.048900,0.025442,0.031804,0.031135,0.022860
100007,4,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,0,0.035088,0.023555,0.043556,0.039186,0.060946,0.035167,0.033131


In [66]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a,set3x
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,-1.732065,1.0,-0.577531,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,...,-0.627631,0.0,-0.502076,1.896448,1.301762,1.758129,1.262144,1.746196,1.488601,1.888223
100003,-1.732054,0.0,-0.577531,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,...,-0.627631,0.0,-0.502076,-0.711865,-0.698289,-0.672096,-0.692681,-0.686084,-0.588371,-0.587171
100004,-1.732043,0.0,-0.577531,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,...,-0.627631,0.0,-0.502076,-0.420592,-0.320577,-0.506078,-0.478319,-0.523846,-0.472024,-0.480227
100006,-1.732031,0.0,-0.577531,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,...,1.593292,0.0,-0.502076,-0.516790,-0.506012,-0.336265,-0.558316,-0.502178,-0.515528,-0.589469
100007,-1.732020,0.0,-0.577531,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,...,-0.627631,0.0,-0.502076,-0.471570,-0.575581,-0.394081,-0.413984,-0.188179,-0.473551,-0.481891


In [67]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,100002,-1.732065,1.0,-0.577531,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,...,-0.627631,0.0,-0.502076,1.896448,1.301762,1.758129,1.262144,1.746196,1.488601,1.888223
1,100003,-1.732054,0.0,-0.577531,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,...,-0.627631,0.0,-0.502076,-0.711865,-0.698289,-0.672096,-0.692681,-0.686084,-0.588371,-0.587171
2,100004,-1.732043,0.0,-0.577531,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,...,-0.627631,0.0,-0.502076,-0.420592,-0.320577,-0.506078,-0.478319,-0.523846,-0.472024,-0.480227
3,100006,-1.732031,0.0,-0.577531,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,...,1.593292,0.0,-0.502076,-0.516790,-0.506012,-0.336265,-0.558316,-0.502178,-0.515528,-0.589469
4,100007,-1.732020,0.0,-0.577531,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,...,-0.627631,0.0,-0.502076,-0.471570,-0.575581,-0.394081,-0.413984,-0.188179,-0.473551,-0.481891


In [68]:
### level_0を削除
df_train.drop('level_0', axis=1, inplace=True)
df_test.drop('level_0', axis=1, inplace=True)
df_test.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,100001,NaN,-0.577531,-0.307114,-0.075104,-0.451791,-0.239271,-0.146484,-0.734184,0.024017,...,-0.627631,0.0,-0.502076,-0.419358,-0.450892,-0.402589,-0.457814,-0.317197,-0.328307,-0.469542
1,100005,NaN,-0.577531,-0.637959,-0.934826,-0.671920,-0.970093,1.079102,-0.464476,-0.891113,...,-0.627631,0.0,-0.502076,0.159147,0.319855,0.256510,0.261749,0.678170,0.365058,0.507163
2,100013,NaN,-0.577531,0.313221,0.159594,2.943903,0.247943,-0.127563,-0.916815,-0.886230,...,1.593292,0.0,1.991729,-0.373564,-0.314285,-0.475758,-0.317065,-0.542220,-0.171688,-0.358904
3,100028,NaN,2.192099,1.347112,2.424818,1.511669,2.805819,0.399658,0.472284,0.221558,...,1.593292,0.0,1.991729,-0.455868,-0.475006,-0.337219,-0.486834,-0.424040,-0.434369,-0.505095
4,100038,NaN,0.807284,0.106443,0.065768,0.342100,0.235763,-0.783203,0.686768,0.082153,...,-0.627631,0.0,-0.502076,0.646859,0.902093,1.006761,0.754380,1.230772,1.196224,1.279548


In [69]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 636.96 MB
Memory usage of optimization is 631002440.00 MB
Decreased by -99064263.5%
Memory usage of dataframe is 100.97 MB
Memory usage of optimization is 100217792.00 MB
Decreased by -99257377.1%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,100002,1.0,-0.577637,0.313232,-0.478027,-0.166138,-0.507324,-0.149780,1.506836,0.747070,...,-0.627441,0.0,-0.501953,1.896484,1.301758,1.757812,1.261719,1.746094,1.488281,1.888672
1,100003,0.0,-0.577637,0.933594,1.725586,0.592773,1.599609,-1.252930,-0.166870,0.511719,...,-0.627441,0.0,-0.501953,-0.711914,-0.698242,-0.671875,-0.692871,-0.686035,-0.588379,-0.587402
2,100004,0.0,-0.577637,-0.927246,-1.153320,-1.404297,-1.091797,-0.783203,-0.689453,0.923828,...,-0.627441,0.0,-0.501953,-0.420654,-0.320557,-0.505859,-0.478271,-0.523926,-0.471924,-0.480225
3,100006,0.0,-0.577637,-0.307129,-0.711426,0.177856,-0.653320,-0.928711,-0.680176,-0.280518,...,1.593750,0.0,-0.501953,-0.516602,-0.505859,-0.336182,-0.558105,-0.501953,-0.515625,-0.589355
4,100007,0.0,-0.577637,-0.431152,-0.213745,-0.361816,-0.068726,0.562988,-0.892578,-0.279785,...,-0.627441,0.0,-0.501953,-0.471680,-0.575684,-0.394043,-0.414062,-0.188232,-0.473633,-0.481934


In [70]:
df_test.drop('TARGET', axis = 1, inplace = True)
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,SK_ID_PREV_nunique_x_high_risk,SK_ID_PREV_nunique_x_low_risk,NUM_INSTALMENT_VERSION_min_high_risk,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,100001,-0.577637,-0.307129,-0.075134,-0.451904,-0.239258,-0.146484,-0.734375,0.024017,-0.051636,...,-0.627441,0.0,-0.501953,-0.419434,-0.450928,-0.402588,-0.457764,-0.317139,-0.328369,-0.469482
1,100005,-0.577637,-0.638184,-0.935059,-0.671875,-0.970215,1.079102,-0.464355,-0.891113,-1.172852,...,-0.627441,0.0,-0.501953,0.159180,0.319824,0.256592,0.261719,0.678223,0.364990,0.507324
2,100013,-0.577637,0.313232,0.159546,2.943359,0.247925,-0.127563,-0.916992,-0.886230,0.797852,...,1.593750,0.0,1.992188,-0.373535,-0.314209,-0.475830,-0.317139,-0.541992,-0.171631,-0.358887
3,100028,2.191406,1.346680,2.425781,1.511719,2.806641,0.399658,0.472168,0.221558,0.847656,...,1.593750,0.0,1.992188,-0.455811,-0.475098,-0.337158,-0.486816,-0.424072,-0.434326,-0.504883
4,100038,0.807129,0.106445,0.065796,0.342041,0.235718,-0.783203,0.686523,0.082153,0.279785,...,-0.627441,0.0,-0.501953,0.646973,0.901855,1.006836,0.754395,1.230469,1.196289,1.279297


In [71]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [72]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [75]:
### training keras
# 1層目のモデル
# pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# pred_test_1a, pred_test_1bは、テストデータの予測値
model_1b = Model1Lgb()
pred_train_1b, pred_test_1b = predict_cv(model_1b, x_train, y_train, x_test)

[LightGBM] [Info] Number of positive: 18633, number of negative: 211996
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.232888 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 112891
[LightGBM] [Info] Number of data points in the train set: 230629, number of used features: 508
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080792 -> initscore=-2.431633
[LightGBM] [Info] Start training from score -2.431633
[1]	train's auc: 0.708538	valid's auc: 0.699403
[2]	train's auc: 0.718693	valid's auc: 0.708634
[3]	train's auc: 0.727821	valid's auc: 0.720352
[4]	train's auc: 0.737204	valid's auc: 0.728706
[5]	train's auc: 0.740237	valid's auc: 0.73154
[6]	train's auc: 0.742472	valid's auc: 0.734192
[7]	train's auc: 0.747601	valid's auc: 0.739786
[8]	train's auc: 0.769908	valid's auc: 0.764238
[9]	train's auc: 0.779501	valid's auc: 0.77463
[10]	train's auc: 0.784998	valid's auc: 0.78047
[11]	train's auc: 0.

[161]	train's auc: 0.800359	valid's auc: 0.792322
[162]	train's auc: 0.800383	valid's auc: 0.792309
[163]	train's auc: 0.800426	valid's auc: 0.792331
[164]	train's auc: 0.800479	valid's auc: 0.792329
[165]	train's auc: 0.800514	valid's auc: 0.792331
[166]	train's auc: 0.800551	valid's auc: 0.792323
[167]	train's auc: 0.800597	valid's auc: 0.792342
[168]	train's auc: 0.800667	valid's auc: 0.792392
[169]	train's auc: 0.800695	valid's auc: 0.792386
[170]	train's auc: 0.800742	valid's auc: 0.792424
[171]	train's auc: 0.800792	valid's auc: 0.792429
[172]	train's auc: 0.800834	valid's auc: 0.792441
[173]	train's auc: 0.800855	valid's auc: 0.792406
[174]	train's auc: 0.800906	valid's auc: 0.792416
[175]	train's auc: 0.800954	valid's auc: 0.79242
[176]	train's auc: 0.800999	valid's auc: 0.792455
[177]	train's auc: 0.801025	valid's auc: 0.792436
[178]	train's auc: 0.801064	valid's auc: 0.792432
[179]	train's auc: 0.801129	valid's auc: 0.792462
[180]	train's auc: 0.801178	valid's auc: 0.792466
[

[327]	train's auc: 0.807081	valid's auc: 0.793242
[328]	train's auc: 0.807127	valid's auc: 0.793246
[329]	train's auc: 0.807162	valid's auc: 0.793246
[330]	train's auc: 0.807203	valid's auc: 0.793253
[331]	train's auc: 0.807241	valid's auc: 0.793242
[332]	train's auc: 0.807273	valid's auc: 0.793236
[333]	train's auc: 0.807309	valid's auc: 0.793249
[334]	train's auc: 0.807366	valid's auc: 0.793294
[335]	train's auc: 0.807405	valid's auc: 0.793289
[336]	train's auc: 0.807437	valid's auc: 0.793301
[337]	train's auc: 0.807472	valid's auc: 0.793297
[338]	train's auc: 0.807519	valid's auc: 0.793289
[339]	train's auc: 0.807564	valid's auc: 0.793298
[340]	train's auc: 0.807617	valid's auc: 0.793316
[341]	train's auc: 0.807656	valid's auc: 0.793316
[342]	train's auc: 0.807706	valid's auc: 0.793324
[343]	train's auc: 0.807737	valid's auc: 0.793331
[344]	train's auc: 0.807777	valid's auc: 0.79332
[345]	train's auc: 0.807807	valid's auc: 0.793316
[346]	train's auc: 0.807842	valid's auc: 0.793317
[

[492]	train's auc: 0.813078	valid's auc: 0.793242
[493]	train's auc: 0.813117	valid's auc: 0.793235
[494]	train's auc: 0.813161	valid's auc: 0.793239
[495]	train's auc: 0.813192	valid's auc: 0.793236
[496]	train's auc: 0.813226	valid's auc: 0.793249
[497]	train's auc: 0.813259	valid's auc: 0.793243
[498]	train's auc: 0.81328	valid's auc: 0.79324
[499]	train's auc: 0.813313	valid's auc: 0.793235
[500]	train's auc: 0.813346	valid's auc: 0.79324
[501]	train's auc: 0.813371	valid's auc: 0.793231
[502]	train's auc: 0.813404	valid's auc: 0.793228
[503]	train's auc: 0.813437	valid's auc: 0.793227
[504]	train's auc: 0.81348	valid's auc: 0.793234
[505]	train's auc: 0.813508	valid's auc: 0.793239
[506]	train's auc: 0.81355	valid's auc: 0.793229
[507]	train's auc: 0.813588	valid's auc: 0.793229
[508]	train's auc: 0.813609	valid's auc: 0.793232
[509]	train's auc: 0.813634	valid's auc: 0.793229
[510]	train's auc: 0.813666	valid's auc: 0.793232
[511]	train's auc: 0.813696	valid's auc: 0.793229
[512]

[657]	train's auc: 0.81858	valid's auc: 0.792922
[658]	train's auc: 0.818621	valid's auc: 0.792913
[659]	train's auc: 0.818645	valid's auc: 0.792915
[660]	train's auc: 0.818675	valid's auc: 0.792915
[661]	train's auc: 0.818713	valid's auc: 0.792916
[662]	train's auc: 0.818752	valid's auc: 0.79294
[663]	train's auc: 0.818783	valid's auc: 0.792927
[664]	train's auc: 0.818803	valid's auc: 0.792924
[665]	train's auc: 0.818844	valid's auc: 0.792918
[666]	train's auc: 0.818876	valid's auc: 0.79292
[667]	train's auc: 0.818903	valid's auc: 0.792917
[668]	train's auc: 0.818931	valid's auc: 0.792912
[669]	train's auc: 0.818961	valid's auc: 0.792908
[670]	train's auc: 0.818981	valid's auc: 0.792908
[671]	train's auc: 0.819014	valid's auc: 0.792913
[672]	train's auc: 0.819036	valid's auc: 0.792916
[673]	train's auc: 0.819059	valid's auc: 0.792915
[674]	train's auc: 0.819089	valid's auc: 0.79291
[675]	train's auc: 0.819112	valid's auc: 0.792906
[676]	train's auc: 0.819141	valid's auc: 0.792902
[677

...

[1299]	train's auc: 0.836023	valid's auc: 0.789988
[1300]	train's auc: 0.836049	valid's auc: 0.789985
[1301]	train's auc: 0.836062	valid's auc: 0.789981
[1302]	train's auc: 0.836082	valid's auc: 0.78997
[1303]	train's auc: 0.836112	valid's auc: 0.789971
[1304]	train's auc: 0.836133	valid's auc: 0.789962
[1305]	train's auc: 0.836151	valid's auc: 0.789959
[1306]	train's auc: 0.836181	valid's auc: 0.789959
[1307]	train's auc: 0.836204	valid's auc: 0.789963
[1308]	train's auc: 0.836226	valid's auc: 0.789961
[1309]	train's auc: 0.836246	valid's auc: 0.789962
[1310]	train's auc: 0.836281	valid's auc: 0.789961
[1311]	train's auc: 0.836303	valid's auc: 0.789952
[1312]	train's auc: 0.836326	valid's auc: 0.789959
[1313]	train's auc: 0.836348	valid's auc: 0.789957
[1314]	train's auc: 0.836361	valid's auc: 0.789964
[1315]	train's auc: 0.836391	valid's auc: 0.789963
[1316]	train's auc: 0.836427	valid's auc: 0.789966
[1317]	train's auc: 0.836451	valid's auc: 0.789959
[1318]	train's auc: 0.836471	val

[1464]	train's auc: 0.839878	valid's auc: 0.789785
[1465]	train's auc: 0.839902	valid's auc: 0.789782
[1466]	train's auc: 0.839925	valid's auc: 0.789779
[1467]	train's auc: 0.839946	valid's auc: 0.789774
[1468]	train's auc: 0.839973	valid's auc: 0.789771
[1469]	train's auc: 0.83999	valid's auc: 0.789766
[1470]	train's auc: 0.840023	valid's auc: 0.789762
[1471]	train's auc: 0.840054	valid's auc: 0.789759
[1472]	train's auc: 0.840092	valid's auc: 0.789761
[1473]	train's auc: 0.840107	valid's auc: 0.789759
[1474]	train's auc: 0.840134	valid's auc: 0.789756
[1475]	train's auc: 0.840182	valid's auc: 0.789748
[1476]	train's auc: 0.840228	valid's auc: 0.789746
[1477]	train's auc: 0.840239	valid's auc: 0.789747
[1478]	train's auc: 0.840258	valid's auc: 0.789747
[1479]	train's auc: 0.840282	valid's auc: 0.789744
[1480]	train's auc: 0.840306	valid's auc: 0.789744
[1481]	train's auc: 0.840324	valid's auc: 0.78974
[1482]	train's auc: 0.840339	valid's auc: 0.789739
[1483]	train's auc: 0.840352	vali

[1633]	train's auc: 0.843753	valid's auc: 0.789467
[1634]	train's auc: 0.843779	valid's auc: 0.78946
[1635]	train's auc: 0.843795	valid's auc: 0.789454
[1636]	train's auc: 0.843811	valid's auc: 0.789451
[1637]	train's auc: 0.843829	valid's auc: 0.789451
[1638]	train's auc: 0.843845	valid's auc: 0.789453
[1639]	train's auc: 0.843873	valid's auc: 0.789455
[1640]	train's auc: 0.843895	valid's auc: 0.789474
[1641]	train's auc: 0.843914	valid's auc: 0.789476
[1642]	train's auc: 0.843932	valid's auc: 0.789468
[1643]	train's auc: 0.843949	valid's auc: 0.789469
[1644]	train's auc: 0.843962	valid's auc: 0.789467
[1645]	train's auc: 0.843988	valid's auc: 0.789469
[1646]	train's auc: 0.844009	valid's auc: 0.789474
[1647]	train's auc: 0.844028	valid's auc: 0.78947
[1648]	train's auc: 0.844048	valid's auc: 0.789469
[1649]	train's auc: 0.844077	valid's auc: 0.78948
[1650]	train's auc: 0.844092	valid's auc: 0.789477
[1651]	train's auc: 0.844123	valid's auc: 0.789476
[1652]	train's auc: 0.844143	valid

[1800]	train's auc: 0.847286	valid's auc: 0.789326
[1801]	train's auc: 0.847304	valid's auc: 0.789322
[1802]	train's auc: 0.847322	valid's auc: 0.789314
[1803]	train's auc: 0.847342	valid's auc: 0.789311
[1804]	train's auc: 0.84736	valid's auc: 0.789317
[1805]	train's auc: 0.847373	valid's auc: 0.789318
[1806]	train's auc: 0.847394	valid's auc: 0.789315
[1807]	train's auc: 0.847408	valid's auc: 0.789312
[1808]	train's auc: 0.847432	valid's auc: 0.789315
[1809]	train's auc: 0.847448	valid's auc: 0.789315
[1810]	train's auc: 0.847462	valid's auc: 0.789315
[1811]	train's auc: 0.847482	valid's auc: 0.789316
[1812]	train's auc: 0.847502	valid's auc: 0.789319
[1813]	train's auc: 0.847505	valid's auc: 0.789316
[1814]	train's auc: 0.84752	valid's auc: 0.789316
[1815]	train's auc: 0.847547	valid's auc: 0.789317
[1816]	train's auc: 0.847561	valid's auc: 0.789315
[1817]	train's auc: 0.847586	valid's auc: 0.789316
[1818]	train's auc: 0.847604	valid's auc: 0.789316
[1819]	train's auc: 0.847622	vali

[1962]	train's auc: 0.85058	valid's auc: 0.789295
[1963]	train's auc: 0.850602	valid's auc: 0.789298
[1964]	train's auc: 0.850654	valid's auc: 0.789312
[1965]	train's auc: 0.850672	valid's auc: 0.789317
[1966]	train's auc: 0.850689	valid's auc: 0.789316
[1967]	train's auc: 0.850703	valid's auc: 0.789317
[1968]	train's auc: 0.850715	valid's auc: 0.789317
[1969]	train's auc: 0.850727	valid's auc: 0.789317
[1970]	train's auc: 0.850755	valid's auc: 0.789302
[1971]	train's auc: 0.850786	valid's auc: 0.789296
[1972]	train's auc: 0.850802	valid's auc: 0.789288
[1973]	train's auc: 0.850819	valid's auc: 0.789299
[1974]	train's auc: 0.850841	valid's auc: 0.789293
[1975]	train's auc: 0.850857	valid's auc: 0.789298
[1976]	train's auc: 0.850878	valid's auc: 0.789295
[1977]	train's auc: 0.850898	valid's auc: 0.789292
[1978]	train's auc: 0.850904	valid's auc: 0.789291
[1979]	train's auc: 0.850922	valid's auc: 0.789286
[1980]	train's auc: 0.850935	valid's auc: 0.789286
[1981]	train's auc: 0.850957	val

In [76]:
pred_test_1b

array([0.03343261, 0.09899595, 0.04218512, ..., 0.00764713, 0.02676942,
       0.23312566])

In [77]:
pred_train_1b

array([0.28393539, 0.02306014, 0.0323981 , ..., 0.03235699, 0.11664275,
       0.11908926])

In [78]:
pred_1b = []
for a in np.array(pred_train_1b):
    pred_1b.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1b)

307506

In [80]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = df_train['SK_ID_CURR'].copy().to_list()
train_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'setB1a': pred_1b})
print(train_x_2.shape)
train_x_2.head()

(307506, 2)


,SK_ID_CURR,setB1a
0,100002,0.283935
1,100003,0.023060
2,100004,0.032398
3,100006,0.028579
4,100007,0.023252


In [81]:
train_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/train_setB1a.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [82]:
pred_1b = []
for a in np.array(pred_test_1b):
    pred_1b.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1b)

48744

In [83]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = df_test['SK_ID_CURR'].copy().to_list()
test_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'setB1a': pred_1b})
print(test_x_2.shape)
test_x_2.head()

(48744, 2)


,SK_ID_CURR,setB1a
0,100001,0.033433
1,100005,0.098996
2,100013,0.042185
3,100028,0.039429
4,100038,0.161166


In [84]:
test_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/test_setB1a.csv" # 出力先
    ,index=None                                        # indexの出力有無
)